# Transforming Data

In [1]:
/Casting is a way to convert a value of one type to a compatible type
/Sometimes the conversion is exact, sometimes information is lost

In [4]:
/A type can be specified in three equivalent ways: symbol, char and num
/We are getting ahead of ourselves and loading a csv but ignore the syntax and just view the table for now

show data:("SSSS";enlist",") 0: `:types.csv

type      type symbol type char type num
----------------------------------------
boolean   `boolean    B         1h      
guid      `guid       G         2h      
byte      `byte       X         4h      
short     `short      H         5h      
int       `int        I         6h      
long      `long       J         7h      
real      `real       E         8h      
float     `float      F         9h      
char      `char       C         10h     
symbol    `           S         11h     
timestamp `timestamp  P         12h     
month     `month      M         13h     
date      `date       D         14h     
datetime  `datetime   Z         15h     
timespan  `timespan   N         16h     
minute    `minute     U         17h     
second    `second     V         18h     
time      `time       T         19h     


In [12]:
"Type Operator"

/Returns data type expressed as short.
/It is a feature of q that data type of an atom is negative whereas
    /datattype of a simple list is positive

type 42

type 10 20 30

type 1.1 2.2 3.3

type "z"

type "abc" /Is positive because a string is a simple list of chars

type `A

/Type of any general list is 0h
type (1.1;2;3)

/Type of any dictionary is 99h
type (`a`b`c!1 2 3)

/Type of any table is 98h
type ([] c1:`a`b`c; c2:10 20 30)

/However a keyed in table has the same type as a dict i.e 99h
type ([k:`a`b`c] v:10 20 30)

"Type Operator"


-7h


7h


9h


-10h


...

-11h


0h


99h


98h


99h


In [13]:
/The type of a variable is the type of the value associated with the variable's name.
/So if we reassign a variable to a different type of value
/Its type changes

a: 42
type a

a:"S"
type a

-7h


-10h


In [18]:
"Casting"

/We use the binary operator $
/Right operand is the source value and left operand specifies the target type
/There are three ways to specify the target type:
    /Short type value (type num in table above). No need to put negative sign.
    /Char type value (type char in table above)
    /Symbol type value (type symbol in table above)


7h$42i /Int to long
6h$42 /Long to int
9h$42 /Long to float


/It is more readable to use Char type instead of short type
"j"$42i /Int to long
"i"$42 /Long to int
"f"$42 /Long to float

/It is most readable to use Symbol type instead of char or short type
`long$42i /Int to long
`int$42 /Long to int
`float$42 /Long to float

"Casting"


42


42i


42f


42


42i


42f


42


42i


42f


In [24]:
/Converting chars to ints, we get their ASCII values
`int$"c"

/Likewise, converting ints to char, we get the char with the ASCII
    /value corresponding to the int
`char$99

99i


"c"


In [25]:
/The value of a date is its count of days from millenium so we can cast to and from int

`date$0

`int$2001.01.01

2000.01.01


366i


In [27]:
/The underlying value of a timespan is its count of nanoseconds from midnight
/So we can cast it to and from long

`long$12:00:00.000000000

`timespan$0

43200000000000


0D00:00:00.000000000


In [40]:
"Casts that Narrow"

/Some casst lose information


`long$12.14 /Float to int

/Wider ints to narrower ones
`short$123456

"Casts that Narrow"


12


0Wh


In [44]:
"Casting to Booleans"

/Zeros are False aka 0b and everything else is True aka 1b

`boolean$0
`boolean$0.0

`boolean$123
`boolean$1.23
`boolean$"c"

"Casting to Booleans"


0b


0b


1b


1b


1b


In [48]:
"Extract Constituents from Complex types"

`date$2015.01.02D10:20:30.123456789

`year$2015.01.02D10:20:30.123456789

`month$2015.01.02D10:20:30.123456789

`mm$2015.01.02D10:20:30.123456789

`dd$2015.01.02D10:20:30.123456789

`hh$2015.01.02D10:20:30.123456789

`minute$2015.01.02D10:20:30.123456789

`uu$2015.01.02D10:20:30.123456789

`second$2015.01.02D10:20:30.123456789

`ss$2015.01.02D10:20:30.123456789

"Extract Constituents from Complex types"


2015.01.02


2015i


2015.01m


1i


...

10i


10:20


20i


10:20:30


30i


In [49]:
"Casting Integral infinites"

/When integral infinities are cast to integers of wider type, 
    /they are their underlying bit patterns, reinterpreted. 
/Since these bit patterns are legitimate values for the wider 
    /type, the cast results in a finite value.

`int$0Wh

`long$0Wi

"Casting Integral infinites"


32767i


2147483647


In [56]:
"Coercing Types"

/Casting can be used to coerce type-safe assignment
/We know that assignment into a simple list must strictly match the type

L: 1 2 3 4 5

L[1]: 42h

"Coercing Types"


[0;31mtype[0m: [0;31mtype[0m

In [57]:
/Coerce the type by casting it to that of the target

L[1]: (type L)$42h
L

1 42 3 4 5


In [58]:
/This coercion to the right type is also very useful when appending the list
L,:(type L)$430h
L

1 42 3 4 5 430


In [59]:
/This is the whole reason why a simple list returns type as positive
/So that we can use its type argument to coerce values given the strict
    /types in simple lists

In [64]:
"Cast is Astomic"

/If we apply it to a list, it hapends to all of the atoms in the list

`int$1 2 3 4

/The above example was of cast being atomic on the right operand
/It is also atomic on the left operand

`char`int`float$42

/Cast is also atomic in both operands simultaneously
`char`int`float$42 43 44

/A neat way to use the char type casting
"cif"$42 43 44

"Cast is Astomic"


1 2 3 4i


"*"
42i
42f


"*"
43i
44f


"*"
43i
44f


In [67]:
"Data to Strings"

/Function string can be used on any q entity to produce a text representation
/The result is always a list of chars so for single digitds you will
    /see a singleton list of a char

string 42

string 4 /Singleton list

a:42

string a

f:{x*x}

string f /Literally the function code is returned

string `Hello

"Data to Strings"


"42"


,"4"


"42"


"{x*x}"


"Hello"


In [66]:
/The string function is not atomic
/It is pseudo atomic, in that it recurses into its argumeent and applies to individual atoms

string 1 2 3

/That is why converting a string to a string results in singleton list of each char
string "abc"

,"1"
,"2"
,"3"


,"a"
,"b"
,"c"


In [70]:
"Creating Symbols from Strings"

`$"abc"

/This allows us to add special chars to symbols that would be impossibel when typing out directly
`$"ABD A"

`ABD A /Space does not work, gives error

"Creating Symbols from Strings"


`abc


`ABD A


[0;31mA[0m: [0;31mA[0m

In [75]:
"Parsing Data from Strings"

/$ operator is overloaded to parse strings into data of any type.
/The overload is invoked by using an uppercase type char as target left operand
    /and string on right operand
/If parse cannot be performed, a null of traget type is returned

/Returns int of "4" and "2" as seen below
"i"$"42"
"i"$("4";"2")

/To get it for 42, we need to invoke overload using uppercase

"I"$"42"
"F"$"42"
"F"$"42.0"
"I"$"42.0" /Returns null

"Parsing Data from Strings"


52 50i


52 50i


42i


42f


42f


0Ni


In [82]:
"Date Parsing"

/Date parsing is flexible with respect to the format of the date

"D"$"12.31.2014"

"D"$"12-31-2014"

"D"$"12/31/2014"

"D"$"12/1/2014"

"D"$"2014/12/31"


/The format MM/DD just does not work and returns null
"D"$"31.12.2014"
"D"$"31-12-2014"
"D"$"31/12/2014"
"D"$"2014/31/12"

"Date Parsing"


2014.12.31


2014.12.31


2014.12.31


2014.12.01


2014.12.31


0Nd


0Nd


0Nd


0Nd


In [85]:
"Function Parsing"

/Create a function from a string use the built in value which is the
    /q interpreter or parse which is the parse step of the q interpreter

f: value "{x*x}"
f 2

ff: parse "{x*x}"
ff 2

"Function Parsing"


4


4


In [88]:
"Creating Typed Empty "

/The general empty list has type 0 since it has no items and 
    /therefore affords no canonical way to determine a type.

type ()

/An issue arises when appending an item in place to a general 
    /empty list, Namely, if that item is an atom, the resulting 
    /singleton list is now a simple list of the type of the atom.
/Recall that a general list returns 0h as well
/So if we get a non 0h type value for a list, it is a simple list

L:()
type L
L,:42
type L

L,: 42.4 /Gives error

"Creating Typed Empty "


0h


0h


7h


[0;31mtype[0m: [0;31mtype[0m

In [95]:
/To avoid this, cast the empty list using the name of the desired 
    /type, which makes it an empty simple list of that type.

/This takes out the guessing game. Our first entry maybe a long when we
    /intended it to be alist of floats but adding a long first types it
    /to longs so its better to type it ourselves.

c1:`float$()
c1,:42.0
c1,:98.6
c1

c1,: 44 /Error

42 98.6


[0;31mtype[0m: [0;31mtype[0m

In [104]:
/Recall the take operator
/Left operand is the n values to take from front or back depending on sign
/Negative sign is for taking from the back of the list. May remind you of pop function from python lists
A: 1 2 3 4
2 # A
-2 # A

/If you extract more items than length of list, it just restarts from the beginning of list and continues extracting
12 # A

/This repetition feature is very useful we can use this to get a list of size n of the same value. Eg 10 15s
10#15

/If we take 0 items from a list, we get an empty list that retains the type of the list it is taking from
0#10 20 30
0#10 20 30i

/This yields a clever way to create typed empty lists
0#0i
0#0f
0#`

1 2


3 4


1 2 3 4 1 2 3 4 1 2 3 4


15 15 15 15 15 15 15 15 15 15


`long$()


`int$()


`int$()


`float$()


`symbol$()


In [106]:
"Enumerations"

/To recap: a binary Cast operator $ trasnforms the right operand
    /into a conforming entity of the left operand
/Now we look at casting extended to user deifned target domains,
    /providing a functional version of enumerated types.

"Enumerations"


In [107]:
"Traditional Enumerations"

/In traditional languages, am enumerated type is a way of associating
    /a series of names with a corresponding set of integral values.

"Traditional Enumerations"


In [136]:
"Data Normalization"

/A powerful use of enumeration is normalizing data.

/Data normalization seeks to eliminate duplication, retaining only
    /the minimum required data.

/Lets create a repetitive list
u: `g`aapl`msft`ibm
u
v:16#u
v

/As you can see we have a list of symbols v drawn from a unique list u

/Consider a new list k that represents the position in u of each item of v

k: u?v
k

/Key observation is that u and k together carry the same info as v
/Indeed they can be used to reconstitute v

u[k]

v~u[k]

/So u and k exactly constitute a traditional enumeration
/v is time consuming to search whereas k is integers and is fast and
    /easy to traverse.
/SO we say u and k normalize v. u stores each symbol once and reading
    /and writing the index list k is a block operation that will be
    /very fast

"Data Normalization"


`g`aapl`msft`ibm


`g`aapl`msft`ibm`g`aapl`msft`ibm`g`aapl`msft`ibm`g`aapl`msft`ibm


0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3


`g`aapl`msft`ibm`g`aapl`msft`ibm`g`aapl`msft`ibm`g`aapl`msft`ibm


1b


In [138]:
"Enumerating Symbols"

/Process of converting a list of symbols to the equivalent list of 
    /indices described above is called enumeration.
/It uses another overload of $ with the name of the variable holding 
    /the unique symbols as left operand and a list of symbols drawn 
    /from that domain on the right.

u: `g`aapl`msft`ibm
v:16#u
k: u?v

/Basic form of enumerated symbol is:
`u$v

/You can recovery underlying integer values by casting to int
ev: `u$v
`int$ev

/Check equal
(`int$k)~`int$ev

/We call u the domain
/The projection`u$ is enumeration over u

"Enumerating Symbols"


`u$`g`aapl`msft`ibm`g`aapl`msft`ibm`g`aapl`msft`ibm`g`aapl`msft`ibm


0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3i


1b


In [148]:
"Using Enumerated Symbols"

sym: `g`aapl`msft`ibm`tsla

v: 25#sym

/Enumerated v
ev: `sym$v

/Enumerated v ev can be subsituted fpr original v in nearly all
    /situations
v[3]
ev[3]

v[3]:`aapl
ev[3]:`aapl
v

where v=`g
where ev=`g

v?`aapl
ev?`aapl

v in `g`ibm
ev in `g`ibm

/While the enumerated version is item-wise equal to the original
    /the entries are not identical
/Equal element wise
all v=ev
v=ev

/Entity wise not equal
v~ev

"Using Enumerated Symbols"


`ibm


`sym$`ibm


`g`aapl`msft`aapl`tsla`g`aapl`msft`ibm`tsla`g`aapl`msft`ibm`tsla`g`aapl`msft`..


0 5 10 15 20


...

1000010010100101001010010b


1000010010100101001010010b


1b


1111111111111111111111111b


0b


In [153]:
"Types of Enumeration"

sym: `g`aapl`msft`ibm`tsla

/Type of enumeration
type `sym$25#sym

/The convention of negative for atoms as below and positive for simple
    /lists as in the previous hold true
type `sym$`g

"Types of Enumeration"


20h


-20h


In [157]:
"Updating an Enumerated List"

/The normalization provided by an enumeration reduces updating all
    /occurences of a given value to a single operation.
/This can have significant performance implications for large lists
    /with many repetitions.

/Just updating your domain is enough as ev gets updated

sym: `g`aapl`msft`ibm`tsla
v: 25#sym
ev: `sym$25#sym

sym[0]: `meta
sym
ev

/In contrast updating in v would require changing every occurence
/Dont worry about the syntax right now
@[v; where v=`g; :; `meta]

"Updating an Enumerated List"


`meta`aapl`msft`ibm`tsla


`sym$`meta`aapl`msft`ibm`tsla`meta`aapl`msft`ibm`tsla`meta`aapl`msft`ibm`tsla..


`meta`aapl`msft`ibm`tsla`meta`aapl`msft`ibm`tsla`meta`aapl`msft`ibm`tsla`meta..


In [166]:
"Dynamically Appending to Enumeration Domain"

/An numeration is more complicated than working with denormalized
    /data when you want to add a new value

/We can only add values to a list if that value is in the domain
/So we must first add it to the enumeration domain and then add to
    /our list.

sym: `g`aapl`msft`ibm`tsla
v: 10#sym
ev: `sym$10#sym

/ev,: `meta /Wont work and gives error

sym,: `meta
ev,: `meta /Now it will work since enumeration domain has it
ev

/So this process has an order
/First we need to check if enumerated domain has it or not
/Add to it if it doesnt and then append to our list
/If you do not know the full extend of the enumeration domain
    /you can use an overload of ? to create the domain on the fly.
/The syntax is similar to the enumeration overload version of $
    /where the name of unique list of symbols is left operand and
    /a source symbol as right operand
`sym?`twtr /Adds twtr
`sym?`g /Doesnt do anything since g is already there
sym

/So we can add it to the list and domain dynamically
ev,: `sym?`nvd
ev

"Dynamically Appending to Enumeration Domain"


`sym$`g`aapl`msft`ibm`tsla`g`aapl`msft`ibm`tsla`meta


`sym$`twtr


`sym$`g


`g`aapl`msft`ibm`tsla`meta`twtr


`sym$`g`aapl`msft`ibm`tsla`g`aapl`msft`ibm`tsla`meta`nvd


In [168]:
"Resolving an Enumeration"

/An enumerated symbol can be susbstituted for its equivalent value in
    /most expressions. However, there are some situations in which
    /you need non enumerated values.
/One case is from converting from one enumerated domain to another, 
    /which happens when copying from one kdb+ database to another
    /or merging two databases.

/Given an enuemrated symbol or list you can recover the un-enumerated
    /values by applying the built in value function

sym: `g`aapl`msft`ibm`tsla
v: 10#sym
ev: `sym$10#sym

/Get un-enumerated values
value ev

v~value ev

"Resolving an Enumeration"


`g`aapl`msft`ibm`tsla`g`aapl`msft`ibm`tsla


1b
